# 🎒 Backpack Prediction Challenge

**FEUP 2024/2025 - L.EIC029 IART**

- Bruno Oliveira - 202208700  
- Henrique Fernandes - 202204988  
- Rodrigo Coelho - 202205188  

> Based on Kaggle Playground Season 5, Episode 2  
> April 2025

---

## 📖 Introduction

This project is related to the application of Supervised Learning techniques to real world classification problems. Specifically, we will develop and evaluate machine learning models that accurately predict the price of student backpacks (target variable), based on a variety of input attributes.

For this we will use a labeled dataset from Kaggle which contains different bag characteristics and we want to understand how they influence the price.

[Link to Backpack Prediction Challenge](https://www.kaggle.com/competitions/playground-series-s5e2/data)

We will follow the machine learning pipeline: data preprocessing, problem definition and target identification, model selection and parameter tuning, model training and testing, and result evaluation and comparison.  

---

## ⚙️ Project setup

### 📦 Virtual Environment

In order to setup the project, use the following commands to setup a virtual environment and install the needed dependencies:

In [ ]:
!python3 -m venv .venv
!source .venv/bin/activate
%pip install -r requirements.txt

Once the dependencies are installed, the script below can be used to download the dataset from the Kaggle competition, using your Kaggle account.

<div class="alert alert-block alert-warning">
<b>Warning:</b> Don't forget to download the Kaggle token associated with your account from the <a href="https://www.kaggle.com/settings">Settings page</a>, move it to the current folder and join the <a href="https://www.kaggle.com/competitions/playground-series-s5e2/">Kaggle playground competition</a>.
</div>

In [ ]:
import os
from pathlib import Path

os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)
dataset_zip = data_dir.joinpath("playground-series-s5e2.zip")
dataset_train = data_dir.joinpath("train.csv")
dataset_extra = data_dir.joinpath("training_extra.csv")

if not dataset_train.exists() or not dataset_extra.exists():
    if not dataset_zip.exists():
        print("Dataset zip not found. Downloading from Kaggle...")
        !kaggle competitions download -c playground-series-s5e2
        print("Download complete.")
    else:
        print("Dataset zip already exists.")

    print("Unzipping the dataset...")
    !unzip -o playground-series-s5e2.zip -d data
    !rm playground-series-s5e2.zip
else:
    print("Dataset already exists. Skipping download and extraction.")


### 📂 Loading the Datasets

With the dependencies met and having downloaded the dataset, we can now load it into our environment. The following commands will load the `train.csv` dataset which contains 300000 entries and is used to train the models.

In [ ]:
import pandas as pd

data = pd.read_csv(dataset_train)

data.info()

print("\nFirst 3 rows of the dataset:")
print(data.head(3))

___

## 🔍 Exploratory Data Analysis

### 🌐 Dataset Overview

As mentioned before, the training dataset consists of *300000* rows of data related to *9* backpack features and the **price** (target variable). 

Most of these features are **categorical** (brand, material, size, laptop compartment, waterproof, style and color) while the others are **numerical** (compartments and weight capacity).

In [ ]:
def overview_data(data, title):
    print(title)
    data_shape = data.shape
    print(f"\nNumber of rows: {data_shape[0]}")
    print(f"Number of columns: {data_shape[1]}")
    print("\nData types of each column:")
    print(data.dtypes)
    data.describe().T

overview_data(data, "Training Dataset Overview")

### ❓ Missing Data

Although the challenge states that the training dataset was generated from a deep learning model trained on another dataset, most of the features include some missing values which were probably introduced to challenge the data preprocessing.

The percentage of **missing values for each feature is below 3.5%** and the **overall percentage of rows with at least one missing value is less than 18%** of the entire training dataset. 

In [ ]:
missing_values = pd.DataFrame({
    'Column': data.columns,
    'Missing Train Values': data.isnull().sum().values,
    'Percentage of Missing Train Values': data.isnull().sum().values / len(data) * 100
})

missing_values = missing_values[~missing_values['Column'].isin(['id', 'Price'])]

summary_row = pd.DataFrame([{
    'Column': 'Rows with Missing Values',
    'Missing Train Values': data.isnull().any(axis=1).sum(),
    'Percentage of Missing Train Values': data.isnull().any(axis=1).sum() / len(data) * 100
}])

missing_values = pd.concat([missing_values, summary_row], ignore_index=True)

missing_values

### 🧬 Duplicated Data

Despite having a moderate amount of missing data, there are no duplicated rows in the training dataset. 

In [ ]:
data_duplicates = data.drop('id', axis=1).duplicated().sum()
print(f"Data duplicates: {data_duplicates}")

### 🧮 Distribution of Data

When it comes to data distribution, all columns follow a relatively constant distribution without any specific outliers or imbalances. Although every categorical feature has a (slightly) dominant category, there are no significant differences in terms of the frequencies of each value. This is also the case for the numeric columns where the distribution of the vaule count is approximately constant.

The graphs don't show any major skewness meaning the dataset appears to be well-suited for classification of all types of backpacks without a specific bias from the training dataset.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

categorical_columns = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
numerical_columns = ['Compartments', 'Weight Capacity (kg)', 'Price']

all_columns = categorical_columns + numerical_columns

def plot_columns(data, categorical_columns, numerical_columns, cols=3):
    rows = (len(all_columns) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 4 * rows))
    axes = axes.flatten()

    for i, column in enumerate(categorical_columns):
        sns.countplot(data=data, x=column, order=data[column].value_counts().index, ax=axes[i])
        axes[i].set_title(f"Distribution of {column}")
        axes[i].tick_params(axis='x', rotation=45)

    for i, column in enumerate(numerical_columns, len(categorical_columns)):
        sns.histplot(data[column], bins=30, ax=axes[i])
        axes[i].set_title(f"Distribution of {column}")

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

plot_columns(data, categorical_columns, numerical_columns)
data.describe().T

### 🔗 Data Correlation

The correlation analysis revealed that no strong linear relationships appear to exist between the columns that could influence the target variable and, consequently, the model training.

As it can be seen both from the Categorical vs. Categorical countplots, there are not major correlations between different categrical features of the dataset. A similar conclusion can be taken from the correlation heatmap of the numeric columns.

In [ ]:
# Plotting correlations between categorical features
num_categories = len(categorical_columns)
cols = 3
rows = (num_categories * (num_categories - 1)) // 2
fig, axes = plt.subplots(rows // cols + (rows % cols > 0), cols, figsize=(15, rows * 1.5))

axes = axes.flatten()
plot_index = 0
for i in range(num_categories):
    for j in range(i + 1, num_categories):
        sns.countplot(x=categorical_columns[i], hue=categorical_columns[j], data=data, ax=axes[plot_index])
        axes[plot_index].set_title(f"Countplot of {categorical_columns[i]} vs {categorical_columns[j]}")
        axes[plot_index].tick_params(axis='x', rotation=45)
        plot_index += 1

for i in range(plot_index, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# Plotting correlation heatmap for numerical features
correlation_matrix = data[numerical_columns].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', cbar=True, linewidths=0.5)
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

### Feature Engineering

In [ ]:
new_columns = ['IBrand', 'ISize', 'Has Laptop Compartment', 'Is Waterproof', 'IStyle', 'IColor']
final_columns = ['Compartments', 'Weight Capacity (kg)'] + new_columns

def impute_data(df):
    for col in categorical_columns:
        df[col] = df[col].fillna(df[col].mode()[0])
    for col in numerical_columns:
        df[col] = df[col].fillna(df[col].median())

def encode_data(df):
    for col in ['Brand', 'Material', 'Style', 'Color']:
        df['I' + col] = df[col].astype('category').cat.codes

    df['ISize'] = df['Size'].map({'Small': 1, 'Medium': 2, 'Large': 3})

    df['Has Laptop Compartment'] = df['Laptop Compartment'].map({'Yes': 1, 'No': 0})
    df['Is Waterproof'] = df['Waterproof'].map({'Yes': 1, 'No': 0})

impute_data(data)
encode_data(data)

### Target Definition

In [ ]:
X = data[final_columns]
y = data['Price']

print(X.head())
print(y.head())

___

## Model Building

### Decision Tree Regressors

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
import math

model = DecisionTreeRegressor(random_state=1, max_depth=5)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
mse = -cross_val_score(model, X, y, cv=10, scoring=mse_scorer)

print(f"Avg Root Mean Squared Error: {math.sqrt(np.mean(mse)):.2f}")
print(f"Min Root Mean Squared Error: {math.sqrt(np.min(mse)):.2f}")

In [ ]:
from sklearn.tree import export_graphviz
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(100, 20))
plot_tree(model, filled=True, feature_names=X.columns, fontsize=10, max_depth=5)
plt.title("Decision Tree Visualization")
plt.show()


In [ ]:
# get all the values from hte Brand column
brand_values = train_data['Brand'].unique()
# get average price per brand
avg_price_per_brand = train_data.groupby('Brand')['Price'].mean().sort_values(ascending=False)
avg_price_per_brand
